In [1]:
import pandas as pd
import pickle

import statsmodels.api as sm
from statsmodels.formula.api import glm

from sklearn.preprocessing import StandardScaler


import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

In [97]:
#finalgen_samples = pd.read_csv('../final_gen.csv')['sample_name']
last_gen = pd.read_csv('../key_files/final_gen.csv')['sample_name']
## eliminate site 33 
last_gen = last_gen[~last_gen.str.startswith('33_')]
## eliminate site 33 
last_gen_samples = last_gen.to_list()

clim_sites_during_exp = pd.read_csv('/carnegie/nobackup/scratch/tbellagio/grene/data/bioclimvars_experimental_sites_era5.csv')
#clim_sites_during_exp = pd.read_csv('../key_files/bioclimvars_sites_era5_year_2018.csv')

sites_af = pd.Series(last_gen_samples).str.split('_').str[0].astype(int)

sites_af.name = 'site'

env = sites_af.reset_index().merge(clim_sites_during_exp).drop(['index'],axis=1)

In [98]:
env_variable = env['bio1']

In [99]:
# Standardize the environmental variable
scaler = StandardScaler()
env_variable_scaled = scaler.fit_transform(env_variable.values.reshape(-1, 1))

In [100]:
pd.DataFrame(env_variable_scaled).to_csv('env.csv',index=None)

In [101]:
env_site_scaled = pd.concat([env['site'],pd.Series(env_variable_scaled.flatten())],axis=1)

In [102]:
env_site_scaled.columns = ['site', 'env_scaled']

In [103]:
env_site_scaled.to_csv('env_site_scaled.csv',index=None)

In [158]:
file_path = '../jacknife_lfmm/splits_samples_last_gen_no33.pkl'

# Open and load the .pkl file
with open(file_path, 'rb') as file:
    samples = pickle.load(file)

train = samples[split][0]
test = samples[split][1]

In [159]:
len(train)

336

In [160]:
len(test)

12

In [105]:
len(samples)

30

In [ ]:
run_partition_binomial_reg_208_30_53729

In [163]:
split = 30
partition = 0

In [164]:
train = samples[split][0]
test = samples[split][1]

IndexError: list index out of range

In [108]:
##get the sites to train/test 
sites_u_train = pd.Series(train).str.split('_').str[0].unique()
sites_u_test = pd.Series(test).str.split('_').str[0].unique()
# conver tthem to int for easiest manipulations
sites_u_train_int =  [int(i) for i in sites_u_train ]
sites_u_test_int =  [int(i) for i in sites_u_test ]

In [109]:
env_site_scaled = pd.read_csv('env_site_scaled.csv')
env_variable_train = env_site_scaled[env_site_scaled['site'].isin(sites_u_train_int)]['env_scaled'].reset_index(drop=True)

In [110]:
env_variable_train

0     -0.696482
1     -0.696482
2     -0.696482
3     -0.696482
4     -0.696482
         ...   
331   -0.319168
332   -0.319168
333   -0.319168
334   -0.319168
335   -0.319168
Name: env_scaled, Length: 336, dtype: float64

In [111]:
env_variable_test = env_site_scaled[env_site_scaled['site'].isin(sites_u_test_int)]
env_site_scaled_unique = env_variable_test.drop_duplicates().set_index('site')


In [112]:
partition_0 = pd.read_csv(f'../baypass_lastgen/individual_gfiles_last_gen/partition_{partition}.txt',header=None) 

pickle_file_path = f'../baypass_lastgen/individual_gfiles_last_gen/column_names_partition_{partition}'
with open(pickle_file_path, 'rb') as file:
    data0 = pickle.load(file)

In [113]:
partition_0.columns = data0

In [114]:
## eliminate site 33 

site_33_delete = [i for i in partition_0.columns if i.startswith('33_')]

partition_0 = partition_0.drop(site_33_delete,axis=1)

## eliminate site 33 

In [115]:
sites_u_train

array(['1', '2', '4', '5', '6', '9', '10', '11', '12', '13', '23', '24',
       '25', '26', '27', '28', '32', '37', '42', '43', '45', '46', '48',
       '49', '52', '53', '54', '55', '57'], dtype=object)

In [116]:
## filter only thre training environments 
partition_0 = partition_0[[i for i in partition_0.columns if any(i.startswith(site + '_') for site  in sites_u_train)]]

In [117]:
partition_0

,1_2_2_minor,1_2_2_major,1_2_3_minor,1_2_3_major,1_3_1_minor,1_3_1_major,1_3_4_minor,1_3_4_major,1_3_5_minor,1_3_5_major,...,57_3_12_minor,57_3_12_major,57_3_13_minor,57_3_13_major,57_3_15_minor,57_3_15_major,57_3_16_minor,57_3_16_major,57_3_18_minor,57_3_18_major
0,132,256,77,157,36,12,25,15,50,94,...,0,60,16,66,10,2,4,30,1,31
1,21,367,5,229,0,48,0,40,26,118,...,0,60,25,57,0,12,6,28,7,25
2,48,340,12,222,3,45,0,40,18,126,...,0,60,1,81,0,12,8,26,16,16
3,4,384,18,216,0,48,1,39,12,132,...,0,60,0,82,0,12,3,31,0,32
4,4,384,18,216,0,48,1,39,12,132,...,0,60,0,82,0,12,3,31,0,32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4994,53,335,28,206,3,45,3,37,37,107,...,2,58,3,79,2,10,3,31,0,32
4995,104,284,26,208,6,42,0,40,13,131,...,0,60,1,81,0,12,10,24,10,22
4996,7,381,6,228,1,47,0,40,21,123,...,0,60,1,81,0,12,1,33,0,32
4997,108,280,44,190,6,42,0,40,15,129,...,8,52,35,47,0,12,14,20,10,22


In [118]:
minor_columns = partition_0.filter(like='minor')
major_columns = partition_0.filter(like='major')

In [119]:
# Properly calculate total allele counts by summing corresponding minor and major allele counts
total_allele_counts = minor_columns.values + major_columns.values

# Create a new DataFrame for total counts with proper column names
total_allele_counts_df = pd.DataFrame(total_allele_counts, columns=minor_columns.columns.str.replace('_minor', '_total'))

In [120]:

# Preparing data for binomial regression
# Minor allele counts as the response variable (successes)
minor_alleles = minor_columns.T.reset_index(drop=True)
major_columns = major_columns.T.reset_index(drop=True)

# Total allele counts as the number of trials (total attempts)
total_alleles = total_allele_counts_df.T.reset_index(drop=True)


In [121]:
successes = minor_alleles.iloc[:,1]
failures = major_columns.iloc[:,1]

In [122]:
# Running binomial regression for each SNP separately
coefficients_df = {}

for i in range(minor_alleles.shape[1]):
    # Prepare response (successes, failures) and predictors (environmental variable)
    successes = minor_alleles.iloc[:,i]
    failures = major_columns.iloc[:,i]
    # Set up the binomial regression model
    X = sm.add_constant(env_variable_train)  # Adding constant for intercept
    y = pd.concat([successes,failures],axis=1)
    # Fit the modelb
    model = sm.GLM(y, X, family=sm.families.Binomial())
    result = model.fit()

    # Extract slope (coefficient for environmental variable) and p-value
    slope = result.params[1]  # Coefficient for the environmental variable
    p_value = result.pvalues[1]  # P-value for the environmental variable
    intercept = result.params[0] 
    
    all_sites_pred_proba = []
    for site, row in env_site_scaled_unique.iterrows():
        # Predict the probability using the logistic model
        X_new = np.column_stack((np.ones(1), np.array([row['env_scaled']])))
        predicted_probability = result.predict(X_new)  # This gives the log-odds
        all_sites_pred_proba.append(predicted_probability[0])
    # Append the results to the DataFrame

    all_sites_pred_proba.append(slope)
    all_sites_pred_proba.append(p_value)
    all_sites_pred_proba.append(intercept)
    coefficients_df[i] = all_sites_pred_proba
    
    # Collect resul

In [123]:
coefficients_df = pd.DataFrame(coefficients_df).T

In [124]:
coefficients_df

,0,1,2,3
0,0.292192,-0.001640,9.014179e-01,-0.882467
1,0.088937,0.118349,2.131489e-07,-2.492037
2,0.090116,-0.075113,1.846189e-04,-2.207273
3,0.155991,0.622553,6.167051e-151,-2.558197
4,0.155991,0.622553,6.167051e-151,-2.558197
...,...,...,...,...
4994,0.083247,-0.267757,3.097985e-47,-2.024911
4995,0.147674,0.019180,2.637296e-01,-1.779761
4996,0.160404,0.299115,1.635329e-53,-2.073154
4997,0.185357,-0.075098,5.363838e-07,-1.375539


In [125]:
sites_u_test

array(['60'], dtype=object)

In [126]:
coefficients_df.columns = [sites_u_test[0], 'slope', 'pvalue','intercept']

In [127]:
coefficients_df

,60,slope,pvalue,intercept
0,0.292192,-0.001640,9.014179e-01,-0.882467
1,0.088937,0.118349,2.131489e-07,-2.492037
2,0.090116,-0.075113,1.846189e-04,-2.207273
3,0.155991,0.622553,6.167051e-151,-2.558197
4,0.155991,0.622553,6.167051e-151,-2.558197
...,...,...,...,...
4994,0.083247,-0.267757,3.097985e-47,-2.024911
4995,0.147674,0.019180,2.637296e-01,-1.779761
4996,0.160404,0.299115,1.635329e-53,-2.073154
4997,0.185357,-0.075098,5.363838e-07,-1.375539


In [128]:
import os

In [ ]:
files = os.listdir('../baypass_lastgen/individual_gfiles_last_gen/')

partitions = [int(file.split('_')[1].replace('.txt', '')) for file in files if '.txt' in file]

In [131]:
partitions.sort()

In [132]:
partitions[-1]

209

In [133]:
len(partitions)

210

In [134]:
pwd -P

'/carnegie/nobackup/scratch/tbellagio/gea_grene-net/binomial_regression_lastgen_go'

In [135]:
import random
import subprocess

In [136]:
file_path = '../jacknife_lfmm/splits_samples_last_gen_no33.pkl'

# Open and load the .pkl file
with open(file_path, 'rb') as file:
    samples = pickle.load(file)

In [137]:
len(samples)

30

In [142]:
# Loop through splits 0 to 31
for split in range(len(samples)):
    # Define the folder path
    folder_path = f'results/split_{split}'
    # Create the directory (makedirs allows creating intermediate directories if they don't exist)
    os.makedirs(folder_path, exist_ok=True)

In [143]:
pwd

'/carnegie/nobackup/scratch/tbellagio/gea_grene-net/binomial_regression_lastgen_go'

In [166]:
# create sbatch files to submit on cedar server
shfiles = []
for split in range(19,31):
    for partition in partitions:
        seed = random.randint(1,100000000)
        file = f'shfiles/partition_{partition}_{split}.sh'
        cmd = f'python run_partition_binomial_reg_last_gen.py {partition} {split}'
        text = f'''#!/bin/bash
#SBATCH --job-name=run_partition_binomial_reg{partition}_{split}
#SBATCH --time=1:00:00  # Time limit set to 4 hours
#SBATCH --ntasks=1
#SBATCH --mem-per-cpu=30gb
#SBATCH --output=run_partition_binomial_reg_{partition}_{split}_%j.out
#SBATCH --mail-user=tbellagio@carnegiescience.edu
#SBATCH --mail-type=FAIL

module load python/3.11_conda
conda activate /home/tbellagio/miniforge3/envs/pipeline_snakemake
export LD_LIBRARY_PATH="/home/tbellagio/miniforge3/envs/run_baypass/lib:$LD_LIBRARY_PATH"
cd /carnegie/nobackup/scratch/tbellagio/gea_grene-net/binomial_regression_lastgen_go
{cmd}

'''
        with open(file, 'w') as o:
            o.write("%s" % text)
        shfiles.append(file)

In [152]:
subprocess.run(["sbatch", shfiles[0]], check=True)

Submitted batch job 53309


CompletedProcess(args=['sbatch', 'shfiles/partition_0_18.sh'], returncode=0)

In [153]:
subprocess.run(["sbatch", shfiles[0]], check=True)

Submitted batch job 53310


CompletedProcess(args=['sbatch', 'shfiles/partition_0_18.sh'], returncode=0)

In [167]:
len(shfiles)

2520

In [168]:
## now run the shfiles
for shfile in shfiles:
    # Submit each sbatch script to the SLURM scheduler
    subprocess.run(["sbatch", shfile], check=True)

Submitted batch job 53731
Submitted batch job 53732
Submitted batch job 53733
Submitted batch job 53734
Submitted batch job 53735
Submitted batch job 53736
Submitted batch job 53737
Submitted batch job 53738
Submitted batch job 53739
Submitted batch job 53740
Submitted batch job 53741
Submitted batch job 53742
Submitted batch job 53743
Submitted batch job 53744
Submitted batch job 53745
Submitted batch job 53746
Submitted batch job 53747
Submitted batch job 53748
Submitted batch job 53749
Submitted batch job 53750
Submitted batch job 53751
Submitted batch job 53752
Submitted batch job 53753
Submitted batch job 53754
Submitted batch job 53755
Submitted batch job 53756
Submitted batch job 53757
Submitted batch job 53758
Submitted batch job 53759
Submitted batch job 53760
Submitted batch job 53761
Submitted batch job 53762
Submitted batch job 53763
Submitted batch job 53764
Submitted batch job 53765
Submitted batch job 53766
Submitted batch job 53767
Submitted batch job 53768
Submitted ba

In [238]:
files = os.listdir('../baypass_lastgen/individual_gfiles_last_gen/')
loci_names = [file for file in files if 'loci' in file]

In [169]:
for split in range(len(samples)):
    print(split)
    print(len(os.listdir(f'results/split_{split}')))

0
210
1
210
2
210
3
210
4
210
5
210
6
210
7
210
8
210
9
210
10
210
11
210
12
210
13
210
14
210
15
210
16
210
17
210
18
210
19
210
20
210
21
210
22
210
23
210
24
210
25
210
26
210
27
210
28
210
29
210


In [170]:
for split in range(len(samples)):
    print(split)
    partitions_r = {}
    for i in range(len(partitions)):
        pickle_file_path = f'../baypass_lastgen/individual_gfiles_last_gen/loci_partition_{i}'
        with open(pickle_file_path, 'rb') as file:
            loci_f = pickle.load(file)
        results = pd.read_csv(f'results/split_{split}/partition{i}.csv')
        results['snp_id'] = loci_f
        partitions_r[i] = results
    results = pd.concat(partitions_r).reset_index(drop=True)
    results.to_csv(f'results/split_{split}/binomial_reg_results_last_gen.csv',index=None)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29


In [ ]:
results/split_13/partition199

In [190]:
'results/split_28/partition31.csv'
'results/split_29/partition0.csv

,2,slope,pvalue,intercept,snp_id
0,0.292570,0.006346,6.241305e-01,-0.886166,1_346
1,0.079556,0.109643,9.234524e-07,-2.504191,1_353
2,0.094720,-0.077272,9.061051e-05,-2.217999,1_363
3,0.095485,0.617205,1.452603e-158,-2.562523,1_395
4,0.095485,0.617205,1.452603e-158,-2.562523,1_396
...,...,...,...,...,...
1048630,0.791339,-0.299355,4.098155e-84,1.485358,5_26975078
1048631,0.071677,-0.044816,4.703856e-02,-2.538407,5_26975121
1048632,0.071677,-0.044816,4.703856e-02,-2.538407,5_26975148
1048633,0.465900,0.094076,2.075031e-15,-0.184487,5_26975272


In [90]:
results.to_csv('binomial_reg_results_last_gen.csv',index=None)

In [87]:
len(results['snp_id'].unique())

1048635

In [89]:
1048635 - 1048635

0

In [73]:
len(results)

1055248

In [66]:
all_loci = []
for i in loci_names:
    pickle_file_path = f'../baypass_lastgen/individual_gfiles_last_gen/{i}'
    with open(pickle_file_path, 'rb') as file:
        loci_f = pickle.load(file)
    all_loci.append(loci_f)

In [67]:
flattened_loci = [item for sublist in all_loci for item in sublist]

In [68]:
len(flattened_loci)

1048635

In [74]:
1054574 - 1055248

-674